In [1]:
# Import Python package
import BinhoPulsar
from BinhoPulsar.Pulsar import Pulsar
from BinhoPulsar.commands.system.definitions import *
from BinhoPulsar.commands.i2c.definitions import *

## Getting started

#### 1. List all the Pulsar devices connected to the PC host

The ``BinhoPulsar.getConnectedPulsarDevicesList()`` gets a list of the Pulsar devices plugged into the host PC machine.

In [2]:
BinhoPulsar.getConnectedPulsarDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FD#7&2450afe2&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fd',
  'serial_number': '80F39E5FDBEF2053B16BBC5AB6EFA89',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Pulsar',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': 0}]

#### 2. Create an instance of the Pulsar class

To utilize a Pulsar USB host adapter device, we need to create an instance of the Pulsar class.

In [3]:
# Create a device instance. One instance of the Pulsar class represents a Pulsar USB host adapter device.
pulsar = Pulsar()

#### 3. Open connection to the Pulsar device

The public method ``Pulsar.open()`` establishes the connection with a Pulsar device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Pulsar serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoPulsar.getConnectedPulsarDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Pulsar device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Pulsar device simultaneously.

In [4]:
# Use the method by default to connect to only one Pulsar device.
pulsar.open()

# Otherwise, use the path attribute to identify each Pulsar device. Uncomment the line below and comment the line #2.
# pulsar.open(path='\\\\?\\HID#VID_1FC9&PID_82FD#7&384f89b9&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Pulsar device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Pulsar

To handle responses and notifications from Pulsar, a callback function must be defined and registered. This function will be invoked every time the Pulsar sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(pulsar_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Pulsar.onEvent(callback_function)`` method.

In [5]:
# Define callback function
def callback_function(pulsar_message: dict, system_message: dict) -> None:
    
    if pulsar_message != None:

        # Print message
        print(">> New message from PULSAR:")
        print(pulsar_message)
    
    if system_message != None:

        # Print message
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
# Register callback function
pulsar.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Pulsar from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliary code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 6. Test communication with a basic command such as ``GET USB STRING``

The ``GET USB STRING`` command allows you to retrieve various string values related to the product, such as the manufacturer, product name, serial number, firmware version, and hardware version. This command serves as a basic test to verify the communication and functionality with the Pulsar device. By successfully receiving the expected string values, you can ensure that the communication between the PC host and the Pulsar device is functioning as expected.

In [8]:
# GET USB STRING - Get Manufacturer
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.MANUFACTURER)

# GET USB STRING  - Get Product Name
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.PRODUCT_NAME)

# GET USB STRING  - Get Serial Number
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.SERIAL_NUMBER)

# GET USB STRING  - Firmware Version
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.FW_VERSION)

# GET USB STRING  - Hardware Version
r = pulsar.getUsbString(getId(), GetUsbStringSubCommand.HW_VERSION)

>> New message from PULSAR:
{'id': 1, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 12, 'payload': 'MN-Binho LLC'}
>> New message from PULSAR:
{'id': 2, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 15, 'payload': 'PR-Binho Pulsar'}
>> New message from PULSAR:
{'id': 3, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 35, 'payload': 'SN-80F39E5FDBEF2053B16BBC5AB6EFA89D'}
>> New message from PULSAR:
{'id': 4, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 8, 'payload': 'FW-4.0.0'}
>> New message from PULSAR:
{'id': 5, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 4, 'payload': 'HW-C'}


## I2C Protocol API

The I2C Protocol API methods are described below.

**I2C Bus definition**

The Pulsar device has two I2C buses, A and B. The I2C Bus A corresponds to the Qwiic port, while the I2C Bus B corresponds to the Breakout Board I2C pins. To select the I2C bus to communicate with, the following variable is defined to use in the I2C-related methods. In this demonstration the I2C Bus A is used by connecting the FRAM memory to the Qwiic connector. To use the I2C Bus B modify the defined variable and connect the memory to the pins on the Breakout Board.

In [9]:
I2C_BUS = I2cBus.I2C_BUS_A

``Pulsar.setI2cSpiUartGpioVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the bus in mV, ranging from 1200 mV up to 3300 mV.

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [1200, 3300] mV. It also allows the value 0 mV to power off the output voltage.

In [10]:
pulsar.setI2cSpiUartGpioVoltage(getId(), 3300)

{'module': 1,
 'opcode': 0,
 'message': 'SET I2C SPI UART GPIO VOLTAGE request success'}

>> New message from PULSAR:
{'id': 6, 'command': 'SYS SET I2C SPI UART GPIO VOLTAGE', 'result': 'SUCCESS'}


``Pulsar.i2cControllerInit(id, busId, frequency, pullUpResistorValue)``

This method initializes the Pulsar as an I2C controller.

  - ``id: int`` : A 2-byte integer that represents the transfer ID.
  - ``busId: I2cBus`` : The I2C Bus identifier.
  - ``frequency: int`` : This parameter represents the I2C SCL frequency in Hz. Currently, the minimum allowed value is 100000 Hz and the maximum allowed value is 1000000 Hz.
  - ``pullUpResistorsValue: I2cPullUpResistorsValue`` : This parameter indicates the desire pull up resistors value to be set in the SDA and SCL lines. This parameter can take the following values:

    - ``I2cPullUpResistorsValue.I2C_PULLUP_4kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_2_2kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1_5kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_680Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_470Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_330Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_220Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_150Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_DISABLED``

In [11]:
pulsar.i2cControllerInit(getId(), I2C_BUS, 1000000, I2cPullUpResistorsValue.I2C_PULLUP_DISABLE)

{'module': 1, 'opcode': 0, 'message': 'I2C CONTROLLER INIT request success'}

>> New message from PULSAR:
{'id': 7, 'command': 'I2C CONTROLLER INIT', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A'}


``Pulsar.i2cControllerSetParameters(id, busId, frequency, pullUpResistorValue)``

This method sets the I2C transfers baudrate and pull up resistors value for the SDA and SCL lines.

  - ``id: int`` : A 2-byte integer that represents the transfer ID.
  - ``busId: I2cBus`` : The I2C Bus identifier.
  - ``frequency: int`` : This parameter represents the I2C SCL frequency in Hz. Currently, the minimum allowed value is 100000 Hz and the maximum allowed value is 1000000 Hz.
  - ``pullUpResistorsValue: I2cPullUpResistorsValue`` : This parameter indicates the desire pull up resistors value to be set in the SDA and SCL lines. This parameter can take the following values:

    - ``I2cPullUpResistorsValue.I2C_PULLUP_4kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_2_2kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1_5kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_680Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_470Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_330Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_220Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_150Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_DISABLED``

In [12]:
pulsar.i2cControllerSetParameters(getId(), I2C_BUS, 100000, I2cPullUpResistorsValue.I2C_PULLUP_2_2kOhm)

{'module': 1, 'opcode': 0, 'message': 'I2C SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 8, 'command': 'I2C CONTROLLER SET PARAMETERS', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A'}


``Pulsar.i2cSetPullUpResistors(id, busId, pullUpResistorsValue)``

This method sets the I2C pull up resistors value for the SDA and SCL signals.

  - ``id: int`` : A 2-byte integer that represents the transfer ID.
  - ``busId: I2cBus`` : The I2C Bus identifier.
  - ``pullUpResistorsValue: I2cPullUpResistorsValue`` : This parameter indicates the desire pull up resistors value to be set in the SDA and SCL lines. This parameter can take the following values:

    - ``I2cPullUpResistorsValue.I2C_PULLUP_4kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_2_2kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1_5kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_1kOhm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_680Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_470Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_330Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_220Ohm``
    - ``I2cPullUpResistorsValue.I2C_PULLUP_150Ohm``

In [13]:
pulsar.i2cSetPullUpResistors(getId(), I2C_BUS, I2cPullUpResistorsValue.I2C_PULLUP_4kOhm)

{'module': 1,
 'opcode': 0,
 'message': 'I2C SET PULL UP RESISTORS request success'}

>> New message from PULSAR:
{'id': 9, 'command': 'I2C SET PULLUP RESISTORS', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A'}


``Pulsar.i2cControllerWrite(id, busId, targetAddress, registerAddress, data, isNonStop = False)``

This method is used to request to the Pulsar device to perform an I2C write transfer. The
I2C write transfer starts with a START condition and ends with a STOP condition.

  - ``id: int`` : It is a 2-bytes integer that represent the transfer id.
  - ``busId: I2cBus`` : The I2C Bus identifier.
  - ``targetAddress: int`` : I2C target static address.
  - ``registerAddress: list `` : A Python list that contains the memory/register address of the I2C target's internal memory, to which the data will be written. The list holds bytes, and can have a length from 0 bytes up to 4 bytes. If the list is empty, the Pulsar will ignore it and write only the data payload.
  - ``data: list`` : A Python list that contains the I2C data to be transferred in the I2C write operation. The list holds bytes elements, and the maximum length is 1024 bytes.
  - ``isNonStop: bool`` : (Optional) This parameter indicates if the I2C write transfer is non-stop, meaning that it ends **without** a STOP condition. If the flag is set to False, the transfer ends with STOP; otherwise, if it's True, the transfer is non-stop, without a STOP condition at the end. By default, the flag is set to False indicating the transfer ends with a STOP condition

In [14]:
TARGET_ADDRESS   = 0x50
TARGET_REGISTER_ADDRESS = [0,0]   # [reg, page]  
data = [44 for i in range(1,112)]

pulsar.i2cControllerWrite(getId(), I2C_BUS, TARGET_ADDRESS, TARGET_REGISTER_ADDRESS, data)

{'module': 1, 'opcode': 0, 'message': 'I2C CONTROLLER WRITE request success'}

>> New message from PULSAR:
{'id': 10, 'command': 'I2C CONTROLLER WRITE', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 111}


``Pulsar.i2cControllerRead(id,  busId, targetAddress, requestDataLength, registerAddress: list = [])``

This method is used to request the USB device to perform an I2C read operation. If the parameter `registerAddress` is populated, the read operation is preceded by an I2C write transfer to send the register/memory address from which the data will be read. A R-Start condition is performed between the I2C write and I2C read transfers.

  - ``id: int`` : A 2-byte integer that represent the transaction ID.
  - ``busId: I2cBus`` : The I2C Bus identifier.
  - ``targetAddress: c_uint8`` : I2C target static address.
  - ``requestDataLength: int`` : Length of the data to be read. The maximum value is 1024 bytes.
  - ``registerAddress: list `` : (Optional) A Python list that contains the memory/register address of the I2C target's internal memory, from which the data will be read. The list holds bytes, and it can have a length from 0 bytes up to 4 bytes. If the list is empty, the Pulsar will perform an I2C Read operation without indicating the register address and starts reading from the last position of the previous transfer.

In [15]:
TARGET_ADDRESS =  0x50
READ_LENGTH = 30
# Indicate the register address to perform an I2C Read From operation
TARGET_REGISTER_ADDRESS = [0,0]   # [reg, page]

pulsar.i2cControllerRead(getId(), I2C_BUS, TARGET_ADDRESS, READ_LENGTH, TARGET_REGISTER_ADDRESS)

{'module': 1, 'opcode': 0, 'message': 'I2C CONTROLLER READ request success'}

>> New message from PULSAR:
{'id': 11, 'command': 'I2C CONTROLLER READ', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 30, 'payload': [44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44]}


**Write Non-Stop**

``Pulsar.i2cControllerWrite(id, busId, targetAddress, registerAddress, data, isNonStop = True)``

In this case, the I2C controller write method is used to request the the USB device to perform an I2C write operation that starts with a START condition but the STOP condition is not performed after the last byte. To indicate this, we will set the `isNonStop` flag to True.

In [16]:
TARGET_ADDRESS   = 0x50
TARGET_REGISTER_ADDRESS = [0,0]   # [reg, page]  
data = [55 for i in range(1,112)]

pulsar.i2cControllerWrite(getId(), I2C_BUS, TARGET_ADDRESS, TARGET_REGISTER_ADDRESS, data, isNonStop=True)

{'module': 1, 'opcode': 0, 'message': 'I2C CONTROLLER WRITE request success'}

>> New message from PULSAR:
{'id': 12, 'command': 'I2C CONTROLLER WRITE', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 111}


``Pulsar.i2cControllerRead(id, busId, targetAddress, requestDataLength, registerAddress: list = [])``

To check the correct execution of the ``i2cControllerWrite`` command with non-stop, an ``i2cControllerRead`` command is performed, reading 70 bytes from the slave register [0, 0]

In [17]:
TARGET_ADDRESS =  0x50
READ_LENGTH = 70
TARGET_REGISTER_ADDRESS = [0,0]   # [reg, page]

pulsar.i2cControllerRead(getId(), I2C_BUS, TARGET_ADDRESS, READ_LENGTH, TARGET_REGISTER_ADDRESS)

{'module': 1, 'opcode': 0, 'message': 'I2C CONTROLLER READ request success'}

>> New message from PULSAR:
{'id': 13, 'command': 'I2C CONTROLLER READ', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 70, 'payload': [55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55]}


**Read**

``Pulsar.i2cControllerRead(id, busId, targetAddress, requestDataLength)``

In this case, the I2C controller read method is used to request the USB device to perform an I2C read operation that starts reading at the final position of the previous transfer.

In [18]:
TARGET_ADDRESS =  0x50
READ_LENGTH = 70

pulsar.i2cControllerRead(getId(), I2C_BUS, TARGET_ADDRESS, READ_LENGTH)

{'module': 1, 'opcode': 0, 'message': 'I2C CONTROLLER READ request success'}

>> New message from PULSAR:
{'id': 14, 'command': 'I2C CONTROLLER READ', 'result': 'SUCCESS', 'i2c_bus': 'I2C_BUS_A', 'payload_length': 70, 'payload': [55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 178, 184, 111, 8, 187, 155, 72, 22, 34, 30, 213, 98, 81, 178, 216, 230, 170, 225, 107, 22, 8, 224, 68, 217, 146, 9, 21, 101, 233]}


## Close communication

Use the ``Pulsar.close()`` method to end the communication with the Pulsar device and release the used memory in the background like threads and so on.

In [19]:
# Close the communication with the Pulsar device.
pulsar.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}